In [1]:
# Import the required modules
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import json
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Libraries for helping us with strings
import string
# Regular Expression Library
import re

# Import our text vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


# Import our classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


# Import some ML helper function
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

# Import our metrics to evaluate our model
from sklearn import metrics

# Library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# You may need to download these from nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords = stopwords.words('english')

from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\conor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\conor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\conor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Functions for modifying text
# Lowercase all words
def make_lower(a_string):
    #print(a_string.lower())
    return a_string.lower()

# Remove all punctuation

def remove_punctuation(a_string):    
    a_string = re.sub(r'[^\w\s]',' ',a_string)
    #print(a_string)
    return a_string

def remove_stopwords(a_string):
    # Break the sentence down into a list of words
    words = word_tokenize(a_string)
    #print(words)
    # Make a list to append valid words into
    valid_words = []
    
    # Loop through all the words
    for word in words:
        
        # Check if word is not in stopwords
        if word not in stopwords:
            
            # If word not in stopwords, append to our valid_words
            valid_words.append(word)
    
    # Join the list of words together into a string
    a_string = ' '.join(valid_words)
    #print(a_string)
    return a_string

def text_pipeline(input_string):
    input_string = make_lower(input_string)
    input_string = remove_punctuation(input_string)
    #input_string = lem_with_pos_tag(input_string)
    input_string = remove_stopwords(input_string)    
    return input_string

def link_transform(link):
    link = text_pipeline(link)
    #print(link)
    #predict_link = vectorizer.transform([link])
    #print("weights ",predict_link)
    #pickle.load(open(filename,'rb'))
    return link

In [30]:
# Main Function
# web scrapper
# Enable Performance Logging of Chrome.
desired_capabilities = DesiredCapabilities.CHROME
desired_capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

# Create the webdriver object and pass the arguments
options = webdriver.ChromeOptions()

# Chrome will start in Headless mode
options.add_argument('headless')

# Ignores any certificate errors if there is any
options.add_argument("--ignore-certificate-errors")

# Startup the chrome webdriver with executable path and
# pass the chrome options and desired capabilities as
# parameters.
driver = webdriver.Chrome(executable_path="C:/chromedriver.exe",
                        chrome_options=options,
                        desired_capabilities=desired_capabilities)

# Send a request to the website and let it load
driver.get("https://time.com/6275538/iran-protests-eyewitness-report/")

# Sleeps for 10 seconds
time.sleep(1)

# Gets all the logs from performance in Chrome
logs = driver.get_log("performance")

# Opens a writable JSON file and writes the logs in it
with open("network_log.json", "w", encoding="utf-8") as f:
    f.write("[")

    # Iterates every logs and parses it using JSON
    for log in logs:
        network_log = json.loads(log["message"])["message"]

        # Checks if the current 'method' key has any
        # Network related value.
        if("Network.response" in network_log["method"]
                or "Network.request" in network_log["method"]
                or "Network.webSocket" in network_log["method"]
                or "Network.requestWillBeSent" in network_log["method"]
                or 'Network.requestWillBeSentExtraInfo' in network_log["method"]):

            # Writes the network log to a JSON file by
            # converting the dictionary to a JSON string
            # using json.dumps().
            f.write(json.dumps(network_log)+",")
    f.write("{}]")

print("Quitting Selenium WebDriver")
driver.quit()

# Read the JSON File and parse it using
# json.loads() to find the urls containing images.
json_file_path = "network_log.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    logs = json.loads(f.read())

links = []
# Iterate the logs
for log in logs:

    # Except block will be accessed if any of the
    # following keys are missing.
    try:
        # URL is present inside the following keys
        url = log["params"]["request"]["url"]
        links.append([url,False])
        #print(url)
        
        # Checks if the extension is .png or .jpg
        if url[len(url)-4:] == ".com" or url[len(url)-4:] == ".js":
            #print(url, end='\n\n')
            pass

    except Exception as e:
        pass

C:\Users\conor\AppData\Local\Temp\ipykernel_20032\1179482492.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/chromedriver.exe",
C:\Users\conor\AppData\Local\Temp\ipykernel_20032\1179482492.py:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="C:/chromedriver.exe",
Incompatible release of chromedriver (version 112.0.5615.28) detected in PATH: C:\Users\conor\OneDrive\Desktop\school\ctp\repos\AdBlocker2.0\chromedriver.exe


Quitting Selenium WebDriver


In [31]:
links

[['https://time.com/6275538/iran-protests-eyewitness-report/', False],
 ['https://fonts.googleapis.com/css?family=PT+Serif:400,700|Roboto:300,400,700|Roboto+Condensed:300,400,400i,700|Lora:400,400i,700,700i|Zilla+Slab:400,400i,500,600,700,700i|Oswald:200,%20300,%20400,500,600,700|Permanent+Marker:400&display=swap',
  False],
 ['https://securepubads.g.doubleclick.net/tag/js/gpt.js', False],
 ['https://cdn.jwplayer.com/libraries/1BWlFwPe.js', False],
 ['https://cdn.cookielaw.org/scripttemplates/otSDKStub.js', False],
 ['https://www.googletagmanager.com/gtag/js?id=UA-97981691-1', False],
 ['https://cdn.speedcurve.com/js/lux.js?id=4265275546', False],
 ['https://micro.rubiconproject.com/prebid/dynamic/20996.js', False],
 ['https://c.amazon-adsystem.com/aax2/apstag.js', False],
 ['https://pub.doubleverify.com/dvtag/21226187/DV759520/pub.js', False],
 ['https://assets.time.com/draper/draper-next/0.5.0.js', False],
 ['https://cdn.taboola.com/libtrc/timecom-timecom/loader.js', False],
 ['https

In [5]:
# edit_links = links
# new_links = []
# for link in edit_links:
#     if 'rr' in link[0]:
#         split_link = link[0].split('?')
#         link[0] = split_link[0]
#         new_links.append(link)
#         print(link)
        
    

In [6]:
# for link in edit_links:
#     print(link)

In [7]:
# links = edit_links
# for link in links:
#     print(link)

In [9]:
# df = pd.DataFrame(data=links)
# df = df.rename(columns={0:"link",1:"isad"})
# df.to_excel('data/ad_list_train.xlsx')

In [12]:
# df = pd.DataFrame(data=links)
# df = df.rename(columns={0:"link",1:"isad"})
# df.to_excel('data/ad_list_test.xlsx')

In [29]:
print(len(links))
df = pd.DataFrame(data = links)
df = df.rename(columns={0:"link",1:"isad"})
df.head()
# df.to_excel('data/youtube_list.xlsx')
ad_df = pd.read_excel('data/ad_list_train.xlsx')
ad_df.head()
combined_df = pd.concat([ad_df,df])
combined_df.to_excel('data/ad_list_train.xlsx')

499


C:\Users\conor\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://securepubads.g.doubleclick.net/gampad/ads?pvsid=1014835702035027&correlator=1939969166380332&eid=31073384%2C31074227%2C31074303&output=ldjh&gdfp_req=1&vrg=202305010101&ptt=17&impl=fifs&iu_parts=21801468956%2Ctime%2Chome%2Cleaderboard1%2Crightrail1%2Csponsorshiplogo1%2Cinline1%2Cinline2%2Cinline3&enc_prev_ius=0%2F1%2F2%2F3%2C0%2F1%2F2%2F4%2C0%2F1%2F2%2F5%2C0%2F1%2F2%2F6%2C0%2F1%2F2%2F7%2C0%2F1%2F2%2F8&prev_iu_szs=728x90%7C5x5%7C1x1%2C1x1%7C1x1%2C8x1%7C1x1%2C300x250%7C1x1%2C300x250%7C1x1%2C300x250%7C1x1&ifi=1&adks=556314269%2C3311774202%2C1063770340%2C2087766350%2C114939507%2C894354823&sfv=1-0-40&prev_scp=pos%3Dleaderboard1%26refreshable%3Dtrue%26breakpoints%3D969x499%257C727x179%257C0x0%26responsive%3Dtrue%26sizes%3D728x90%257C970x250%257C5x5%257C1x1%26dv_upt_pid%3D914f6fd1-cfff-4c19-9ef0-0fb54de7a9b2%26dv_upt_sid%3D04bbe1ab-f2ff-487e-848a-1a733e8446f2%26dv_upt_sacnt%3D1%26amznbid

In [32]:
print(len(links))
df = pd.DataFrame(data = links)
df = df.rename(columns={0:"link",1:"isad"})
df.head()
# df.to_excel('data/youtube_list.xlsx')
ad_df = pd.read_excel('data/ad_list_test.xlsx')
ad_df.head()
combined_df = pd.concat([ad_df,df])
combined_df.to_excel('data/ad_list_test.xlsx')

557


C:\Users\conor\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://pub.doubleverify.com/dvtag/signals/vlp/pub.json?ctx=21226187&cmp=DV759520&url=https%3A%2F%2Ftime.com%2F6275538%2Firan-protests-eyewitness-report&vlp=1&tvp=1&slot-0-21801468956%2Ftime%2Fworld%2Fleaderboard1=728x90%2C970x250%2C5x5%2C1x1%2C728x90%2C5x5%2C1x1&slot-1-21801468956%2Ftime%2Fworld%2Fsponsorshiplogo1=8x1%2C1x1&slot-2-21801468956%2Ftime%2Fworld%2Finline1=970x250%2C728x90%2C300x250%2C336x280%2C1x1%2C300x250%2C300x600%2C336x280%2C1x1%2C300x250%2C300x600%2C336x280%2C1x1&slot-3-21801468956%2Ftime%2Fworld%2Finline2=970x250%2C728x90%2C300x250%2C336x280%2C1x1%2C300x250%2C300x600%2C336x280%2C1x1%2C300x250%2C300x600%2C336x280%2C1x1&slot-4-21801468956%2Ftime%2Fworld%2Finline3=970x250%2C728x90%2C300x250%2C336x280%2C1x1%2C300x250%2C300x600%2C336x280%2C1x1%2C300x250%2C300x600%2C336x280%2C1x1&slot-5-21801468956%2Ftime%2Fworld%2Finline4=970x250%2C728x90%2C300x250%2C336x280%2C1x1%2C300x250

In [72]:
# testing model prediction
filename = 'data/trained_vectorizer.pkl'
model = pickle.load(open(filename,'rb'))
print(type(model))
filename = 'data/trained_vectorizer.pkl'
vectorizer = pickle.load(open(filename,'rb'))
print(type(vectorizer))
for link in links:
    clean_link = link_transform(link[0])
    predict_link = vectorizer.transform([clean_link])
    print(clean_link, " predicted ", model.predict(predict_link))


<class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
https www youtube com watch v owivjmgms9e  predicted  [False]
https www youtube com desktop fa273944 jsbin web animations next lite min vflset web animations next lite min js  predicted  [False]
https www youtube com desktop fa273944 jsbin webcomponents nopatch vflset webcomponents nopatch js  predicted  [False]
https www youtube com desktop fa273944 jsbin fetch polyfill vflset fetch polyfill js  predicted  [False]
https www youtube com desktop fa273944 jsbin intersection observer min vflset intersection observer min js  predicted  [False]
https ytimg com generate_204  predicted  [False]
https www youtube com desktop fa273944 jsbin scheduler vflset scheduler js  predicted  [False]
https www youtube com desktop fa273944 jsbin www i18n constants en_us vflset www i18n constants js  predicted  [False]
https fonts googleapis com css2 family roboto wght 300 400 500 700 family youtube sans wg